# Estimate Binomial draw probabilility

In [1]:
using StatisticalRethinking
using DynamicHMC, TransformVariables, LogDensityProblems, MCMCDiagnostics
using Parameters, ForwardDiff

┌ Info: Recompiling stale cache file /Users/rob/.julia/compiled/v1.2/DynamicHMC/iPqvI.ji for DynamicHMC [bbc10e6e-7c05-544b-b16e-64fede858acb]
└ @ Base loading.jl:1187
┌ Info: Recompiling stale cache file /Users/rob/.julia/compiled/v1.2/MCMCDiagnostics/kMBOZ.ji for MCMCDiagnostics [6e857e4b-079a-58c4-aeab-bc2670384359]
└ @ Base loading.jl:1187


Define a structure to hold the data.

In [2]:
struct BernoulliProblem
    "Total number of draws in the data."
    n::Int
    "Number of draws `==1` in the data"
    s::Vector{Int}
end;

Make the type callable with the parameters *as a single argument*.

In [3]:
function (problem::BernoulliProblem)((α, )::NamedTuple{(:α, )})
    @unpack n, s = problem        # extract the data
    loglikelihood(Binomial(n, α), s)
end

Create the data and complete setting up the problem.

In [4]:
obs = rand(Binomial(9, 2/3), 1)
p = BernoulliProblem(9, obs)
p((α = 0.5, ))

-1.8075078261961934

Use a flat priors (the default, omitted) for α

In [5]:
P = TransformedLogDensity(as((α = as𝕀,)), p)
∇P = ADgradient(:ForwardDiff, P);

Sample

In [6]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000)

MCMC, adapting ϵ (75 steps)
0.0012 s/step ...done
MCMC, adapting ϵ (25 steps)
8.9e-6 s/step ...done
MCMC, adapting ϵ (50 steps)
0.0031 s/step ...done
MCMC, adapting ϵ (100 steps)
1.4e-5 s/step ...done
MCMC, adapting ϵ (200 steps)
7.9e-6 s/step ...done
MCMC, adapting ϵ (400 steps)
6.3e-6 s/step ...done
MCMC, adapting ϵ (50 steps)
9.3e-6 s/step ...done
MCMC (1000 steps)
1.9e-5 s/step ...done


(DynamicHMC.NUTS_Transition{Array{Float64,1},Float64}[NUTS_Transition{Array{Float64,1},Float64}([0.901596], -3.18963, 1, DoubledTurn, 0.924245, 1), NUTS_Transition{Array{Float64,1},Float64}([-0.0144489], -3.62766, 1, AdjacentTurn, 0.897547, 3), NUTS_Transition{Array{Float64,1},Float64}([0.377614], -3.1087, 1, AdjacentTurn, 1.0, 3), NUTS_Transition{Array{Float64,1},Float64}([1.07066], -3.32377, 1, AdjacentTurn, 0.908093, 3), NUTS_Transition{Array{Float64,1},Float64}([1.16334], -3.26036, 1, DoubledTurn, 0.974491, 1), NUTS_Transition{Array{Float64,1},Float64}([0.958163], -3.16838, 1, DoubledTurn, 1.0, 1), NUTS_Transition{Array{Float64,1},Float64}([1.05756], -3.10408, 1, DoubledTurn, 0.976253, 1), NUTS_Transition{Array{Float64,1},Float64}([1.42101], -3.63355, 1, DoubledTurn, 0.890885, 1), NUTS_Transition{Array{Float64,1},Float64}([0.304806], -3.98705, 1, AdjacentTurn, 0.947307, 3), NUTS_Transition{Array{Float64,1},Float64}([1.05662], -4.05211, 2, DoubledTurn, 0.787776, 3)  …  NUTS_Transiti

To get the posterior for ``α`` use `get_position` and then transform back.

In [7]:
posterior = TransformVariables.transform.(Ref(∇P.transformation), get_position.(chain));

Extract the parameter.

In [8]:
posterior_α = first.(posterior);

check the effective sample size

In [9]:
ess_α = effective_sample_size(posterior_α)

412.5021680536656

NUTS-specific statistics

In [10]:
NUTS_statistics(chain)

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.92, min/25%/median/75%/max: 0.23 0.89 0.97 1.0 1.0
  termination: AdjacentTurn => 35% DoubledTurn => 65%
  depth: 1 => 64% 2 => 36%


check the mean

In [11]:
mean(posterior_α)

0.6302511171203886

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*